# Find the Repos Available in your Database, and What Repository Groups They Are In

## Connect to your database

In [7]:
import psycopg2
import pandas as pd 
import sqlalchemy as salc
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
import datetime
import json
warnings.filterwarnings('ignore')

with open("config.json") as config_file:
    config = json.load(config_file)

database_connection_string = 'postgres+psycopg2://{}:{}@{}:{}/{}'.format(config['user'], config['password'], config['host'], config['port'], config['database'])

dbschema='augur_data'
engine = salc.create_engine(
    database_connection_string,
    connect_args={'options': '-csearch_path={}'.format(dbschema)})

### Retrieve Available Respositories

In [8]:
repolist = pd.DataFrame()

repo_query = salc.sql.text(f"""
             SELECT a.rg_name,
                a.repo_group_id,
                b.repo_name,
                b.repo_id,
                b.forked_from,
                b.repo_archived 
            FROM
                repo_groups a,
                repo b 
            WHERE
                a.repo_group_id = b.repo_group_id 
            ORDER BY
                rg_name,
                repo_name;   

    """)

repolist = pd.read_sql(repo_query, con=engine)

display(repolist)

repolist.dtypes

,rg_name,repo_group_id,repo_name,repo_id,forked_from,repo_archived
0,babel,25430,,25481,Parent not available,0.0
1,babel,25430,acorn-to-esprima,25453,Parent not available,1.0
2,babel,25430,actions,25484,Parent not available,0.0
3,babel,25430,awesome-babel,25461,Parent not available,0.0
4,babel,25430,babel,25430,Parent not available,0.0
...,...,...,...,...,...,...
363,withastro,25431,snowpack-plugin-starter-template,25503,Parent not available,0.0
364,withastro,25431,standard-pkg,25492,Parent not available,0.0
365,withastro,25431,vercel-custom-log-drain,25505,Parent not available,0.0
366,withastro,25431,vite,25515,vitejs/vite,0.0


rg_name           object
repo_group_id      int64
repo_name         object
repo_id            int64
forked_from       object
repo_archived    float64
dtype: object

### Create a Simpler List for quickly Identifying repo_group_id's and repo_id's for other queries

In [3]:
repolist = pd.DataFrame()

repo_query = salc.sql.text(f"""
             SELECT b.repo_id,
                a.repo_group_id,
                b.repo_name,
                a.rg_name
            FROM
                repo_groups a,
                repo b 
            WHERE
                a.repo_group_id = b.repo_group_id 
            ORDER BY
                rg_name,
                repo_name;   

    """)

repolist = pd.read_sql(repo_query, con=engine)

display(repolist)

repolist.dtypes

,repo_id,repo_group_id,repo_name,rg_name
0,25440,25155,augur,chaoss
1,25460,25155,augur-license,chaoss
2,25558,25155,community-reports,chaoss
3,25449,25155,governance,chaoss
4,25448,25155,grimoirelab,chaoss
...,...,...,...,...
75,30356,60017,salt-winrepo-ng,SaltStack
76,30362,60017,salty-vagrant,SaltStack
77,30347,60017,sublime-text,SaltStack
78,30380,60017,takara,SaltStack


repo_id           int64
repo_group_id     int64
repo_name        object
rg_name          object
dtype: object